### Imports

In [1]:
### Standard imports
import pandas as pd
import numpy as np

### Plotting imports
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()

### Removes warnings that occassionally show in imports
import warnings
warnings.filterwarnings('ignore')

# Helps convert String representation of list into a list
import ast

### Helper functions

In [2]:
def string_literal(x):
    if x is np.nan:
        return np.nan
    else:
        try:
            return ast.literal_eval(x)
        except:
            return x

### Load data

In [ ]:
test = pd.read_csv("/PBS_1-100.csv")
for col in ["Transcript", "Story", "Speakers"]:
    test[col] = test[col].map(string_literal)